In [1]:
import networkx as nx
import axelrod as axl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import random
%matplotlib inline
from networkx.drawing.nx_agraph import graphviz_layout
#import statsmodels.formula.api as sm

In [11]:
strategies = [s for s in axl.ordinary_strategies if not s().classifier['stochastic']] 
players = [s() for s in strategies]

In [12]:
def Neighborhood(G):
    Neighborhood = []
    for i in range(len(players)) : Neighborhood.append(G.neighbors(i))
    return Neighborhood

In [13]:
def Neighborhood_size (Neighborhood) :
    Neighborhood_size = []
    for i in range(len(Neighborhood)) : Neighborhood_size.append(len(Neighborhood[i]))
    return Neighborhood_size

In [14]:
def NeighborsScores(G, results):
    return [[results.scores[j] for j in G.neighbors(i)]
            for i, pl in enumerate(players)]

In [15]:
def AvNeighborsScore(G, results) :
    av_score = []
    temp = [[j for m in G.neighbors(k) for j in results.scores[m]]
              for k in G.nodes()]
    for i in G.nodes() :
        av_score.append(np.mean(temp[i]))
    return av_score

In [19]:
def touranment(G, p, seed):
    """Run a random tournament with a binomial 
       structure.
    """
    random.seed(seed)
    np.random.seed(seed)
    edges = G.edges()
    tournament = axl.SpatialTournament(players, edges=G.edges(), turns = 200, repetitions=5)
    return tournament.play(processes=0,
                           filename="/home/nikoleta/src/Dissertation-Notebooks/Data/Binomial/Interactions/Binomial_Tournament{}.csv".format(p, seed))

In [20]:
def tournament_results(G, seed,p):
    results = touranment(G, p, seed)
    neighborhood = Neighborhood(G)
    nsize = Neighborhood_size(neighborhood)
    Degree = list(G.degree(G.nodes()).values())
    nscores = NeighborsScores(G, results)
    av_nscores = AvNeighborsScore(G, results)
    
    data = pd.DataFrame({'seed' : i, 'Parameter': p ,'PlayersIndex' : G.nodes(),
                         'PlayersName' : results.players, 'Degree' : Degree ,
                         'Neighborhood' : neighborhood, 'NeighborhoodSize' : nsize,
                         'Ranking' : results.ranking, 'Scores' : results.scores,
                         'NormalisedScores' : results.normalised_scores,
                         'AverageScore' : [np.median(scores) for scores in results.normalised_scores],
                         'NeighborsScore' : nscores, 'AverageNeighborsScore' : av_nscores,
                         #'Payoffs' : tournament.game.RPST(),
                         'Connectivity' : nx.node_connectivity(G), 
                         'Clustering' : nx.average_clustering(G)

                        })
    return data

In [21]:
results = pd.DataFrame()
for seed in range(0, 1):
    
    np.random.seed(seed)
    G = nx.binomial_graph(len(players), .5)
    p = seed/100
    results = results.append([tournament_results(G, seed,p)])

results.to_csv("/home/nikoleta/src/Dissertation-Notebooks/Data/Binomial/Binomial.csv", index=False) 

Analysing results:   0%|          | 0/19 [00:00<?, ?it/s]

IndexError: list index out of range